<a href="https://colab.research.google.com/github/BVika/Methods_of_semantic_information_processing/blob/main/%D0%9B%D0%B0%D0%B1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Задания

1. Применить к текстам лемматизацию, удаление стоп слов и токенизацию по словам
2. Реализовать Bag of Words
3. Реализовать TF-IDF

## Загрузка библиотек

In [7]:
pip install nltk

In [8]:
pip install pymorphy3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 27.9 MB/s eta 0:00:00


### **Задание 1**
*Применить к текстам лемматизацию, удаление стоп слов и токенизацию по словам*

# `на русском`


Импорт библиотек

In [9]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pymystem3 import Mystem
from typing import List
import pymorphy3

In [10]:
# Пример текста
texts = [
    "Я люблю программирование и изучение новых технологий.",
    "Машинное обучение - это увлекательная область."
]

In [11]:
# Загрузка необходимых ресурсов
nltk.download('punkt_tab')
nltk.download('stopwords')

# Инициализация морфологического анализатора
m = Mystem()  # Программа MyStem производит морфологический анализ текста на русском языке

# Функция для лемматизации
def lemmatize(text: List[str]) -> List[str]:
    lemmas = []
    for word in text:
        lemmas.extend(m.lemmatize(word))  # Лемматизируем каждое слово
        cleaned_lemmas = [lemma.strip() for lemma in lemmas if lemma.strip() and lemma not in [' ']]
    return cleaned_lemmas


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [12]:
# Удаление стоп-слов и лемматизация для каждого текста
for text in texts:
    # Токенизация
    tokens = word_tokenize(text)
    # Удаление стоп-слов
    stop_words = set(stopwords.words('russian'))
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    # Лемматизация
    lemmas = lemmatize(filtered_tokens)

    # Результат
    print(lemmas)

['любить', 'программирование', 'изучение', 'новый', 'технология', '.']
['машинный', 'обучение', '-', 'это', 'увлекательный', 'область', '.']


# `на английском`



Импорт библиотек

In [13]:
from nltk.stem import WordNetLemmatizer

In [14]:
# Пример текста
texts = [
    "I love programming and learning new technologies.",
    "Machine learning is an exciting field."
]

In [15]:
# Загрузка необходимых ресурсов
nltk.download('punkt_tab')
nltk.download('wordnet')

# Инициализация лемматизатора
lemmatizer = WordNetLemmatizer()

# Функция для лемматизации
def lemmatize(text: List[str]) -> List[str]:
    lemmas = [lemmatizer.lemmatize(word) for word in text]  # Лемматизируем каждое слово
    return lemmas




[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [16]:
# Удаление стоп-слов и лемматизация для каждого текста
for text in texts:
    # Токенизация
    tokens = word_tokenize(text)
    # Удаление стоп-слов
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    # Лемматизация
    lemmas = lemmatize(filtered_tokens)

    # Результат
    print(lemmas)

['love', 'programming', 'learning', 'new', 'technology', '.']
['Machine', 'learning', 'exciting', 'field', '.']


### **Задание 2**
Реализовать Bag of Words

In [17]:
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from collections import Counter
import numpy as np

In [18]:
texts = [
    "Природа полна удивительных чудес, и в каждом чуде природы скрыта своя красота.",
    "Леса и горы дарят нам ощущение спокойствия и гармонии, которые можно найти только в природе.",
    "Птицы поют свои мелодии, и их мелодии наполняют воздух радостью и счастьем.",
    "Река текет, отражая солнечные лучи и зелень вокруг, создавая волшебный мир природы.",
    "Каждый сезон приносит свои краски и запахи, и каждый сезон природы уникален.",
    "Цветы распускаются весной, радуя глаз яркими оттенками, и весна пробуждает природу.",
    "Закат над озером создает волшебную атмосферу и умиротворение, которое ощущается в природе.",
    "Прогулка по лесу позволяет ощутить силу и красоту природы, которая окружает нас повсюду."
]

In [19]:
def preprocessing(texts: list[str]) -> list[list[str]]:
    """Предобработка текста, возвращающая список списков токенов."""
    tokenized_texts = [text.lower().replace(',', '').replace('.', '').split() for text in texts]  # Токенизация на слова
    return tokenized_texts

def make_dict(texts: list[list[str]]) -> dict[str, int]:
    """Создание словаря уникальных слов из текстов."""
    unique_words = set()
    for text in texts:
        unique_words.update(text)  # Добавление уникальных слов в множество
    return {word: index for index, word in enumerate(unique_words)}  # Присвоение индексов

def count_num_words(texts: list[list[str]], word_dict: dict[str, int]) -> list[list[int]]:
    """Подсчет количества вхождений слов для каждого текста."""
    word_counts = []
    for text in texts:
        count_vector = [0] * len(word_dict)  # Инициализация вектора счетчиков
        for word in text:
            if word in word_dict:
                count_vector[word_dict[word]] += 1
        word_counts.append(count_vector)
    return word_counts


# Токинизация
tokenized_texts = preprocessing(texts)

# Создание словаря уникальных слов
word_dict = make_dict(tokenized_texts)

# Подсчет количества слов
word_counts = count_num_words(tokenized_texts, word_dict)

# Результат
print("Словарь уникальных слов:", word_dict)
print("Матрица Bag of Words:", word_counts)


Словарь уникальных слов: {'нам': 0, 'каждом': 1, 'уникален': 2, 'пробуждает': 3, 'природе': 4, 'весна': 5, 'прогулка': 6, 'полна': 7, 'по': 8, 'волшебную': 9, 'красоту': 10, 'нас': 11, 'найти': 12, 'гармонии': 13, 'в': 14, 'лучи': 15, 'радостью': 16, 'наполняют': 17, 'солнечные': 18, 'природы': 19, 'оттенками': 20, 'создавая': 21, 'сезон': 22, 'и': 23, 'позволяет': 24, 'радуя': 25, 'воздух': 26, 'мелодии': 27, 'ощущение': 28, 'отражая': 29, 'вокруг': 30, 'запахи': 31, 'спокойствия': 32, 'яркими': 33, 'приносит': 34, 'река': 35, 'силу': 36, 'только': 37, 'ощущается': 38, 'леса': 39, 'текет': 40, 'волшебный': 41, 'повсюду': 42, 'можно': 43, 'над': 44, 'своя': 45, 'которые': 46, 'распускаются': 47, 'дарят': 48, 'глаз': 49, 'их': 50, 'каждый': 51, 'которая': 52, 'умиротворение': 53, 'чудес': 54, 'создает': 55, 'зелень': 56, 'цветы': 57, 'природу': 58, 'окружает': 59, 'которое': 60, 'счастьем': 61, 'свои': 62, 'ощутить': 63, 'птицы': 64, 'поют': 65, 'горы': 66, 'атмосферу': 67, 'природа': 6

### **Задание 3**
Реализовать TF-IDF

Частотность термина (TF). Измеряет, насколько часто слово встречается в документе. Рассчитывается как отношение количества вхождений слова к количеству слов в документе.

Обратная частотность документа (IDF). Измеряет, насколько редким является слово в наборе документов. Рассчитывается как логарифм отношения количества документов в коллекции к количеству документов в коллекции, в которых встречается заданное слово.

Общий показатель TF-IDF является произведением TF и IDF. Формула: TF-IDF = TF * log(IDF).

In [29]:
import math
from typing import List, Dict
from nltk.corpus import stopwords

In [30]:
def preprocessing(texts: List[str]) -> List[List[str]]:
    """Предобработка текста, удаляющая стоп-слова и знаки препинания, возвращающая список списков токенов."""
    punctuation_marks = ['!', ',', '(', ')', ':', '-', '?', '.', '..', '...', '«', '»', ';', '–', '--']
    stop_words = set(stopwords.words("russian"))
    tokenized_texts = []
    for text in texts:
        # Приводим текст к нижнему регистру
        text = text.lower()
        # Удаляем знаки препинания
        for punctuation in punctuation_marks:
            text = text.replace(punctuation, '')
        # Токенизируем текст
        tokens = text.split()
        # Удаляем стоп-слова
        filtered_tokens = [word for word in tokens if word not in stop_words]
        tokenized_texts.append(filtered_tokens)
    return tokenized_texts


# Предобработка
tokenized_texts = preprocessing(texts)
print(tokenized_texts)

[['природа', 'полна', 'удивительных', 'чудес', 'каждом', 'чуде', 'природы', 'скрыта', 'своя', 'красота'], ['леса', 'горы', 'дарят', 'нам', 'ощущение', 'спокойствия', 'гармонии', 'которые', 'найти', 'природе'], ['птицы', 'поют', 'свои', 'мелодии', 'мелодии', 'наполняют', 'воздух', 'радостью', 'счастьем'], ['река', 'текет', 'отражая', 'солнечные', 'лучи', 'зелень', 'вокруг', 'создавая', 'волшебный', 'мир', 'природы'], ['каждый', 'сезон', 'приносит', 'свои', 'краски', 'запахи', 'каждый', 'сезон', 'природы', 'уникален'], ['цветы', 'распускаются', 'весной', 'радуя', 'глаз', 'яркими', 'оттенками', 'весна', 'пробуждает', 'природу'], ['закат', 'озером', 'создает', 'волшебную', 'атмосферу', 'умиротворение', 'которое', 'ощущается', 'природе'], ['прогулка', 'лесу', 'позволяет', 'ощутить', 'силу', 'красоту', 'природы', 'которая', 'окружает', 'повсюду']]


In [31]:
def make_dict(texts: List[List[str]]) -> Dict[str, int]:
    """Создание словаря уникальных слов из текстов."""
    unique_words = set()
    for text in texts:
        unique_words.update(text)
    return {word: index for index, word in enumerate(unique_words)}

# Создание словаря уникальных слов
word_dict = make_dict(tokenized_texts)

In [32]:
def compute_tf(texts: List[List[str]], word_dict: Dict[str, int]) -> List[List[float]]:
    """Подсчет TF (Term Frequency) для каждого текста."""
    tf_matrix = []
    for text in texts:
        count_vector = [0] * len(word_dict)
        for word in text:
            if word in word_dict:
                count_vector[word_dict[word]] += 1
        # Нормируем частоты
        total_words = len(text)
        tf_vector = [count / total_words for count in count_vector]
        tf_matrix.append(tf_vector)
    return tf_matrix

# Подсчет TF
tf_matrix = compute_tf(tokenized_texts, word_dict)
print(tf_matrix)

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.1, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.1, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1], [0.0, 0.2222222222222222, 0.1111111111111111, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1111111111111111, 0.0, 0.0, 0.1111111111111111, 0.1111111111111111, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1111111111111111,

In [33]:
def compute_idf(texts: List[List[str]], word_dict: Dict[str, int]) -> List[float]:
    """Подсчет IDF (Inverse Document Frequency) для слов."""
    idf_vector = []
    num_docs = len(texts)
    for word in word_dict.keys():
        count = sum(1 for text in texts if word in text) #возвращает количество документов, в которых присутствует данное слово. Если слово не встречается ни в одном документе, count будет равно 0.
        idf = math.log(num_docs / (count + 1))  # Добавляем 1 для избежания деления на ноль
        idf_vector.append(idf)
    return idf_vector
# Подсчет IDF
idf_vector = compute_idf(tokenized_texts, word_dict)
print(idf_vector)

[1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 0.9808292530117262, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 0.9808292530117262, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 0.47000362924573563, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906, 1.3862943611198906

In [34]:
def compute_tfidf(tf_matrix: List[List[float]], idf_vector: List[float]) -> List[List[float]]:
    """Подсчет TF-IDF на основе TF и IDF."""
    tfidf_matrix = []
    for tf_vector in tf_matrix:
        tfidf_vector = [tf * idf for tf, idf in zip(tf_vector, idf_vector)]
        tfidf_matrix.append(tfidf_vector)
    return tfidf_matrix
# Подсчет TF-IDF
tfidf_matrix = compute_tfidf(tf_matrix, idf_vector)
print(tfidf_matrix)

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.13862943611198905, 0.0, 0.0, 0.0, 0.0, 0.13862943611198905, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.13862943611198905, 0.13862943611198905, 0.0, 0.0, 0.0, 0.0, 0.13862943611198905, 0.0, 0.0, 0.13862943611198905, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.04700036292457357, 0.13862943611198905, 0.0, 0.0, 0.0, 0.0, 0.0, 0.13862943611198905, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.13862943611198905, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.13862943611198905, 0.13862943611198905, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.13862943611198905, 0.0, 0.0, 0.09808292530117263, 0.0, 0.0, 0.0, 0.13862943611198905, 0.0, 0.0, 0.0, 0.0, 0.0, 0.13862943611198905, 0.0, 0.0, 0.13862943611198905, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.13862943611198905, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.13862943611198905, 0.0, 0.0, 0.

In [35]:
# Результат
print("Словарь уникальных слов:", word_dict)
print("Матрица TF-IDF:", tfidf_matrix)

Словарь уникальных слов: {'которое': 0, 'мелодии': 1, 'счастьем': 2, 'нам': 3, 'ощущение': 4, 'красота': 5, 'глаз': 6, 'отражая': 7, 'свои': 8, 'ощутить': 9, 'каждом': 10, 'птицы': 11, 'поют': 12, 'уникален': 13, 'горы': 14, 'пробуждает': 15, 'вокруг': 16, 'природе': 17, 'весна': 18, 'запахи': 19, 'прогулка': 20, 'спокойствия': 21, 'атмосферу': 22, 'полна': 23, 'природа': 24, 'волшебную': 25, 'красоту': 26, 'найти': 27, 'каждый': 28, 'удивительных': 29, 'гармонии': 30, 'лучи': 31, 'чуде': 32, 'яркими': 33, 'озером': 34, 'приносит': 35, 'мир': 36, 'река': 37, 'радостью': 38, 'наполняют': 39, 'солнечные': 40, 'краски': 41, 'силу': 42, 'природы': 43, 'скрыта': 44, 'которая': 45, 'ощущается': 46, 'леса': 47, 'оттенками': 48, 'умиротворение': 49, 'чудес': 50, 'текет': 51, 'создавая': 52, 'сезон': 53, 'создает': 54, 'закат': 55, 'волшебный': 56, 'зелень': 57, 'повсюду': 58, 'весной': 59, 'цветы': 60, 'природу': 61, 'лесу': 62, 'своя': 63, 'которые': 64, 'окружает': 65, 'позволяет': 66, 'раду